In [33]:
import os
import sys
import time
import napari
import pickle
import h5py
import numpy as np
import pandas as pd

from skimage import measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import white_tophat, disk

from napari.qt.threading import thread_worker
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

from itertools import product

from tifffile import imread,imsave,TiffFile
import matplotlib.pyplot as plt
import seaborn as sn

from nd2reader import ND2Reader
import btrack

In [35]:
myFov = '05'

myDirIm = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tiffs'
myFile_track_im = r'20210618_RPE_p21_cycD1_DHB_H2B_series_05_ch_01.tif'
myFile_signal_im = r'20210618_RPE_p21_cycD1_DHB_H2B_series_05_ch_02.tif'

myDirTracks = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking\05'
dfDir = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking\05_napari'

In [9]:
try:
    os.mkdir(pointDir)
    print('Directory for saving created.')
except(FileExistsError):
    print('Directory exists.')

Directory exists.


In [10]:
modelPath = r'D:\BARC\Sonja_2021\b_track\cell_config.json'

btrack_init_path = os.path.join(pointDir,f'{myFov}_tracks_init.h5')
btrack_mod_path = os.path.join(pointDir,f'{myFov}_tracks_mod.h5')

## Read in the tracking channel

In [11]:
%%time

# read in image

myIm = imread(os.path.join(myDirIm,myFile_track_im))
myIm.shape

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


Wall time: 1min 34s


(361, 2765, 2765)

## Read in signal channels

In [14]:
%%time

# read in image

myIm_signal = imread(os.path.join(myDirIm,myFile_signal_im))
myIm_signal.shape

Wall time: 51.8 s


(361, 2765, 2765)

## Read in tracks in a form of labels

In [16]:
# read in labels as tracks

myLabels =[]

fovFiles = [x for x in os.listdir(myDirTracks) if f'series_{myFov}_' in x]

for myFile in fovFiles:
    
    # read a mask
    labels = imread(os.path.join(myDirTracks,myFile))
    
    myLabels.append(labels)
    
inProgressTracks = np.array(myLabels)

## Get latest tracking

In [17]:
# check if modified tracking exists

try:
    f = open(btrack_mod_path.replace('h5','pkl'),'rb')
    
    print('Opening corrected tracking.')

except(FileNotFoundError):
  
    f = open(btrack_init_path.replace('h5','pkl'),'rb')
    print('Opening original tracking.')

data, properties, graph = pickle.load(f)

Opening original tracking.


## Get the full data frame

In [94]:
cellDataAll=pd.read_pickle(os.path.join(dfDir,f'cellPose_btrack_regionprops_{myFov}.pkl'))
cellDataAll.drop(['file','x','y'],axis=1,inplace=True)

In [95]:
# create an array visualization of accepted points
selData=cellDataAll.loc[cellDataAll.accepted==True,:]
acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

In [97]:
cellDataAll

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,bbox-3,image,mean_intensity-0,mean_intensity-1,t,track_id,parent,generation,root,accepted
0,1,618,32.343042,1976.189320,-0.955054,31.727605,24.907658,19,1961,46,1992,"[[False, False, False, False, False, False, Fa...",809.165049,364.655340,0,28.0,28.0,0.0,28.0,False
1,2,667,47.545727,260.788606,-0.441942,37.601948,22.681589,31,249,66,275,"[[False, False, False, False, False, False, Fa...",898.422789,532.869565,0,40.0,40.0,0.0,40.0,False
2,3,830,77.783133,1890.766265,0.375525,42.503848,24.970511,58,1877,98,1907,"[[False, False, False, False, False, False, Fa...",848.244578,403.140964,0,77.0,77.0,0.0,77.0,False
3,4,900,81.102222,953.490000,-0.335190,38.246862,30.045222,63,938,101,969,"[[False, False, False, False, False, False, Fa...",987.763333,405.835556,0,76.0,76.0,0.0,76.0,False
4,5,593,93.485666,69.274874,1.451172,33.456928,22.639231,82,53,105,86,"[[False, False, False, False, False, False, Fa...",899.733558,440.807757,0,75.0,75.0,0.0,75.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46044,138,859,2644.790454,685.828871,0.436377,36.452341,30.066009,2628,671,2663,702,"[[False, False, False, False, False, False, Fa...",735.294529,2467.003492,360,1081.0,1081.0,0.0,1081.0,False
46045,139,581,2645.559380,60.783133,0.530008,33.697577,22.014273,2631,49,2661,74,"[[False, False, False, False, True, True, True...",742.841652,2534.953528,360,577.0,577.0,0.0,577.0,False
46046,140,1185,2664.096203,2710.520675,-1.263715,47.913538,31.553204,2648,2688,2681,2735,"[[False, False, False, False, False, False, Fa...",756.792405,689.045570,360,1134.0,1134.0,0.0,1134.0,False
46047,141,1307,2711.799541,579.492731,0.965957,52.616304,31.707215,2692,557,2732,603,"[[False, False, False, False, False, False, Fa...",785.349656,1304.887529,360,1095.0,1095.0,0.0,1095.0,False


## Visualize

In [131]:
viewer = napari.Viewer()

# layer
viewer.add_image(myIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1)
viewer.add_image(myIm_signal,colormap='red',contrast_limits=(0, 2000),opacity = 1,visible=False)

# layer
track_layer=viewer.add_tracks(data, properties=properties, graph=graph,name='tracking')
track_layer.display_id=True

viewer.add_points([],name='modPoints',face_color='red',ndim=3)
layer_accepted = viewer.add_points(acceptedPoints,name='accepted objects',face_color='green', opacity =0.5, ndim=3)

# layer 
layer_inProgress = viewer.add_labels(inProgressTracks,name='objects',opacity = 0.5)

In [66]:
# right click for accepting points
# toggles status of a track and visualizes it in the accepted objects layer if changed into accepted

@layer_inProgress.mouse_drag_callbacks.append
def accept_track(layer, event):

    if(event.button == 2):
        
        # look up cursor position
        x = int(viewer.cursor.position[1])
        y = int(viewer.cursor.position[2])
        
        # get data
        inProgressTracks = layer_inProgress.data
        
        # check which cell was clicked
        myTrackNum = inProgressTracks[viewer.dims.current_step[0],x,y]
        
        if myTrackNum > 0:

            # check status
            trackStatus = list(cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'])[0]

            # change status of this track
            cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'] = not(trackStatus)

            # regenerate accepted points
            selData=cellDataAll.loc[cellDataAll.accepted==True,:]
            acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

            # update viewer    
            layer_accepted.data = acceptedPoints

In [135]:
# transfer modification of labels 
# -> into tracks layer
# -> into cellDataAll

# at the moment - triggered by 'u' key on labels layer

regProps = ['label', 'area','centroid','major_axis_length','minor_axis_length','bbox','image','mean_intensity']

@viewer.bind_key('u',overwrite=True)
def update_inProgress(viewer):
    
    global cellDataAll
    
    data = viewer.layers['tracking'].data
    properties = viewer.layers['tracking'].properties

    # get the positionu
    myT = viewer.dims.current_step[0]

    # create intensity image
    intIm = np.moveaxis([myIm[myT,:,:],myIm_signal[myT,:,:],inProgressTracks[myT,:,:]],0,2)

    # define objects to change 
    changeIndex = (data[:,1]==myT)

    # find new features of all the objects in the frame
    cellData = pd.DataFrame(measure.regionprops_table(inProgressTracks[myT,:,:], properties=regProps, intensity_image=intIm))
    cellData['t'] = myT
    cellData['track_id'] = cellData['mean_intensity-2'].astype(int) # measurement directly from the labels layer
    cellData.drop('mean_intensity-2',axis=1,inplace=True)

    # find info for labels if existing

    # create a unique data frame
    info_data = cellDataAll.loc[:,['track_id','parent','root','generation','accepted']].drop_duplicates()

    # merge it to the data of this frame
    cellData = cellData.merge(info_data,on='track_id')

    # swap in the general data frame
    cellDataAll.drop(cellDataAll[cellDataAll.t==myT].index,axis=0,inplace=True)
    cellDataAll = cellDataAll.append(cellData,ignore_index=True)

    # modify data of the track layer
    frameData = np.array(cellData.loc[:,['label','t','centroid-0','centroid-1']])

    data = np.delete(data,changeIndex,axis=0)

    data = np.vstack([data, frameData])

    # modify properties of the track layer

    cellData['state'] = 5

    for tProp in properties.keys():

        properties[tProp] = np.delete(properties[tProp],changeIndex)
        properties[tProp] = np.append(properties[tProp], cellData[tProp])

    # change tracks layer
    viewer.layers['tracking'].data = data
    viewer.layers['tracking'].properties = properties

    # change status
    viewer.status = f'Frame {myT} was modified.' 

    
# -> for changed traces re-generate small movie

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [132]:
# swapping tracks based on 4 points

@viewer.bind_key('r',overwrite=True)
def swap_points(viewer):

    swapPoints = viewer.layers['swapPoints'].data.astype(int)

    # check that you get a good set of swapPoints
    lenTest = (len(swapPoints) == 4)

    if lenTest:

        framesToSwap = list(set(swapPoints[:,0]))
        framesToSwap.sort()
        frameTest = (framesToSwap[1] == framesToSwap[0]+1)


        # read in in-progress tracks
        inProgressTracks = viewer.layers['inProgressStack'].data

        # check tracks
        myTracks = []
        for myPoint in swapPoints:

            myTracks.append(inProgressTracks[tuple(myPoint)])

        trackTest = (len(set(myTracks)) == 2)

        if (lenTest and frameTest and trackTest):

            # get tracks numbers
            track_1 = list((set(myTracks)))[0] 
            track_2 = list((set(myTracks)))[1]

            frame_change = np.max(framesToSwap)


            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

            # change labels layer
            viewer.layers.pop('inProgressStack')
            viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)


            # change graph
            children_1 = set(data[((properties['parent']==track_1) & (properties['generation']>0)),0])

            for child in children_1:

                graph[int(child)] = track_2

            children_2 = set(data[((properties['parent']==track_2) & (properties['generation']>0)),0])

            for child in children_2:

                graph[int(child)] = track_1


            # change tracking data
            track1_change = ((data[:,0]==track_1) & (data[:,1]>=frame_change))
            track2_change = ((data[:,0]==track_2) & (data[:,1]>=frame_change))

            data[track2_change,0] = track_1
            data[track1_change,0] = track_2

            # change info about the parent 
            track1_change = ((properties['parent']==track_1) & (data[:,1]>=frame_change))
            track2_change = ((properties['parent']==track_2) & (data[:,1]>=frame_change))

            properties['parent'][track1_change] = track_2
            properties['parent'][track2_change] = track_1

            # change info about the root
            root1 = properties['root'][track1_change][0]
            root2 = properties['root'][track2_change][0]

            track1_change = ((properties['root']==root1) & (data[:,1]>=frame_change))
            track2_change = ((properties['root']==root2) & (data[:,1]>=frame_change))

            properties['root'][track1_change] = root2
            properties['root'][track2_change] = root1

            # change info about the generation
            gen1 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_1)]
            gen2 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_2)]

            properties['generation'][track1_change] = properties['generation'][track1_change] + gen2 - gen1
            properties['generation'][track2_change] = properties['generation'][track2_change] + gen1 - gen2



            # update tracking data
            viewer.layers['data'].data = data
            viewer.layers['data'].properties = properties
            viewer.layers['data'].graph = graph

            # clean swap points
            viewer.layers['swapPoints'].data = []

            viewer.status='Tracks have been swapped.'

        else:

            viewer.status='Swap points are incorrect.'
    else:

        viewer.status='Swap points are incorrect.'

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [ ]:
# add a button to Napari interface

In [68]:
@track_layer.bind_key('s',overwrite=True)
def update_inProgress(viewer):

    viewer_small = napari.Viewer()
    myTrack = 1 
    
    # create gallery
    
    viewer_small.add_image(myGallery,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

In [137]:
%%time

# rewrite it as a function that can work with an arbitrary number of signal channels

# create gallery
# there is an option to cut with or without surronding (at the moment it's with)

myTrack = 1
imSize = 100

data = track_layer.data

# calculate how many images there will be
imNum = np.sum(data[:,0]==myTrack)

# cut images to a small stack
# here deal with edge effects
smallIm = np.zeros([imNum,imSize,imSize]).astype('uint16')
smallLabel = smallIm.copy()

startFrame = int(np.min(data[data[:,0]==myTrack,1]))

for myInd in np.where(data[:,0]==myTrack)[0]:
    
    x = int(data[myInd,2])
    y = int(data[myInd,3])
    myFrame = int(data[myInd,1])
    
    t = myFrame - startFrame
    
    smallIm[t,:,:] = myIm[myFrame,x-int(imSize/2):x+int(imSize/2),y-int(imSize/2):y+int(imSize/2)]
    smallLabel[t,:,:] = inProgressTracks[myFrame,x-int(imSize/2):x+int(imSize/2),y-int(imSize/2):y+int(imSize/2)]
    
# make gallery out of stack

if np.sqrt(imNum).is_integer():
    
    col = row = int(np.sqrt(imNum))

else:
    
    col = int(np.sqrt(imNum))
    row = int(np.floor(imNum/col)+1)
            
# create canvas
myGallery = np.zeros([row*imSize,col*imSize]).astype('uint16')
myGalleryLabel = myGallery.copy()

# put images into canvas
for pair in product(range(row),range(col)):
        
        i = pair[1] # column iterator (fast)
        j = pair[0] # row iterator (slow)
        
        myFrame = j*col+i
        
        if myFrame<imNum:
        
            myGallery[j*imSize:(j+1)*imSize,i*imSize:(i+1)*imSize] = smallIm[myFrame,:,:] 
            myGalleryLabel[j*imSize:(j+1)*imSize,i*imSize:(i+1)*imSize] = smallLabel[myFrame,:,:]
        else:
            break
            
myGallery = [myGallery,myGalleryLabel]

Wall time: 84 ms


In [167]:
myTrack = 1

myGraph_data = cellDataAll.loc[cellDataAll.track_id==myTrack,['t','mean_intensity-0']] 

In [168]:
mpl_widget = FigureCanvas(Figure(figsize=(2, 1)))
static_ax = mpl_widget.figure.subplots()
static_ax.scatter(myGraph_data.t,myGraph_data['mean_intensity-0'])

viewer_gal.window.add_dock_widget(mpl_widget)

In [164]:
myGraph_data

,t,mean_intensity-1
65,0,756.410795
168,1,747.205090
274,2,777.085072
380,3,684.698941
488,4,675.072727
...,...,...
45485,358,2036.917638
45630,359,2126.119876
45774,360,2276.289243
45828,97,1152.741160


In [145]:
try:
    viewer_stack.close()
except:
    pass

In [209]:
try:
    viewer_stack.close()
except:
    pass

viewer_stack = napari.Viewer()

viewer_stack.add_image(smallIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1,name=f'track {myTrack}')
viewer_stack.add_labels(smallLabel,name='inProgressStack',opacity = 0.5)

<Labels layer 'inProgressStack' at 0x2338c820048>

In [157]:
def display_gallery(myGallery):
    
    global viewer_gal

    try:
        viewer_gal.close()
    except:
        pass

    viewer_gal = napari.Viewer()

    viewer_gal.add_image(myGallery[0],colormap='viridis',contrast_limits=(0, 2000),opacity = 1)
    viewer_gal.add_labels(myGallery[1],name='inProgressStack',opacity = 0.5)

In [165]:
display_gallery(myGallery)

In [ ]:
# create a button (or a whole menu) for these gallery views
# add numbering of the frames to gallery
# add graphs to other viewers

In [ ]:
# what to do with ghost objects

In [199]:
data[data[:,0]==1092,:]

array([[1092.        ,  318.        , 2162.92535971, 2026.29406475],
       [1092.        ,  319.        , 2162.92535971, 2026.29406475],
       [1092.        ,  320.        , 2139.88372093, 2058.0455814 ]])

In [201]:
data[(data[:,0]==7) & (data[:,1]==318),:]

array([[   7.        ,  318.        , 2172.5382482 , 2015.13455069]])

In [ ]:
# for faster checking of traces you need a galery
# the other option would be also to look at a trace, best DHB

# small movies should be generated only for traces longer than n frames

In [ ]:
# key binding for linking function

# check if parent have different offspring

# no other offspring
# connect points

# other offspring
# connect
# add to the graph

# what if objects didn;t